In [ ]:
#

In [ ]:
#db_connection

db_connection = mysql.connector.connect(
    host="68.178.155.255",           # or IP address
    user="Anika12",
    password="Anika12",
    database="categories_lead"
)
cursor = db_connection.cursor(dictionary=True)
cursor.execute("SHOW TABLES;")
for table in cursor:
  print(table)


In [ ]:
!pip install mysql-connector-python -q
import mysql.connector


!pip install transformers accelerate -q

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_id = "google/flan-t5-xl"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)


In [ ]:
def   build_negative_sentiment_flag_prompt(conversation):
    return f"""
You are a CRM assistant.

Your task is to **analyze this lead conversation** and check if it shows **any sign of negative sentiment**, based on these 3 types:

🔹 Complaint – about service, delays, lack of follow-up, or experience
🔹 Frustration – irritated tone, repeated follow-ups, impatience
🔹 Negative Product/Service Feedback – bugs, crashes, not working, too expensive

✅ If **any one** of the above is present in the conversation, return: **True**
❌ If none are found, return: **False**

---

Examples:

1.
sent: Just checking again.
received: You guys never reply on time.
→ Complaint detected
Return: True

2.
sent: Hi, any update?
received: This tool is crashing again.
→ Negative Product Feedback detected
Return: True

3.
sent: Hello, following up.
received: No one listens!
→ Frustration detected
Return: True

4.
sent: Thanks, we’ll get back if needed.
received: Okay, noted.
→ No negative signal
Return: False

---

Now analyze the conversation below and return only `True` or `False`:

{conversation}

Answer:
"""


In [ ]:
def run_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


In [ ]:
#updates both inactivity_days and emails_category

from datetime import datetime
import mysql.connector

# =========================
# DB CONNECTION
# =========================
db_connection = mysql.connector.connect(
    host="68.178.155.255",
    user="Anika12",
    password="Anika12",
    database="categories_lead"
)
cursor = db_connection.cursor(dictionary=True)

# =========================
# MAIN PROCESS
# =========================
cursor.execute("SELECT DISTINCT lead_id FROM emails WHERE lead_id IS NOT NULL")
lead_ids = cursor.fetchall()

for row in lead_ids:
    lead_id = row['lead_id']

    # Get last_contact_date from lead_engagement_summary
    cursor.execute("""
        SELECT last_contact_date
        FROM lead_engagement_summary
        WHERE lead_id = %s
    """, (lead_id,))
    contact_row = cursor.fetchone()
    last_contact_date = contact_row['last_contact_date'] if contact_row else None

    # Step 1: Calculate inactivity_days
    inactivity_days = 0
    if last_contact_date:
        inactivity_days = (datetime.now().date() - last_contact_date).days
        print(f"📅 Lead {lead_id} → Days since last contact: {inactivity_days}")
    else:
        print(f"⚠️ Lead {lead_id} has no last_contact_date")

    # Step 2: Analyze last 5 emails for sentiment
    cursor.execute("""
        SELECT direction, body, timestamp
        FROM emails
        WHERE lead_id = %s
        ORDER BY timestamp DESC
        LIMIT 5
    """, (lead_id,))
    latest_emails = cursor.fetchall()
    email_chain = list(reversed(latest_emails))  # oldest to newest

    print(f"\n===== LEAD {lead_id} — Last 5 Email Pairs Analysis =====")
    negative_count = 0
    i = 0
    while i < len(email_chain) - 1:
        current = email_chain[i]
        next_msg = email_chain[i + 1]

        if current['direction'].lower() == 'sent' and next_msg['direction'].lower() == 'received':
            sent_text = current['body'].strip()
            received_text = next_msg['body'].strip()
            pair_convo = f"sent: {sent_text}\nreceived: {received_text}"
            prompt = build_negative_sentiment_flag_prompt(pair_convo)
            output = run_model(prompt)

            print(f"📨 Sent: {sent_text}")
            print(f"📨 Received: {received_text}")
            print(f"🧠 Model Output: {output}")

            if output.lower() == "true":
                negative_count += 1
                print("🔴 Negative Sentiment Detected!")
                print("------------------------------------------------")

            i += 2
        else:
            i += 1

    # Step 3: Update lead_engagement_summary
    cursor.execute("""
        UPDATE lead_engagement_summary
        SET emails_category = %s,
            inactivity_days = %s
        WHERE lead_id = %s
    """, (negative_count, inactivity_days, lead_id))

    print(f"🟡 Lead {lead_id} ➤ Sentiment Count (emails_category): {negative_count}, Inactivity Days: {inactivity_days}")

# =========================
# FINALIZE
# =========================
db_connection.commit()
cursor.close()
db_connection.close()
print("\n✅ All updates complete in lead_engagement_summary.")


In [ ]:
# uodates for calls
from datetime import datetime
import mysql.connector
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# =========================
# MODEL SETUP
# =========================
model_name = "google/flan-t5-base"  # You can change to flan-t5-xl if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

def run_model(prompt):
    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=10)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        return result if result else "False"
    except Exception as e:
        print(f"⚠️ Model error: {e}")
        return "False"

# =========================
# PROMPT FUNCTION
# =========================
def build_negative_sentiment_flag_prompt_for_calls(latest_call_result_block):
    return f"""
You are a CRM assistant.

Your task is to **analyze this latest call summary** and check if it shows **any sign of negative sentiment**, based on these 3 types:

🔹 Complaint – about service, delays, lack of follow-up, or poor experience  
🔹 Frustration – irritated tone, repeated follow-ups, impatience  
🔹 Negative Product/Service Feedback – bugs, integration issues, not working, too expensive

✅ If **any one** of the above is present in the call summary, return: **True**  
❌ If none are found, return: **False**

---

Examples:

1.  
📞 call_result: "Pending"  
📝 description: "Client is annoyed by delays and said no one followed up"  
→ Complaint & Frustration  
Return: True

2.  
📞 call_result: "Resolved"  
📝 description: "Client complained about confusing integration options"  
→ Negative Product Feedback  
Return: True

3.  
📞 call_result: "Negotiation"  
📝 description: "Lead is pushing for big discounts, unhappy with pricing"  
→ Negative Product Feedback (Price)  
Return: True

4.  
📞 call_result: "Connected"  
📝 description: "Client asked some clarifications, no concerns raised"  
→ No negative sentiment  
Return: False

---

Now analyze the call summary below and return only `True` or `False`:

📞 call_result: {latest_call_result_block.get('call_result', 'N/A')}  
📝 description: {latest_call_result_block.get('description', 'N/A')}

Answer:
"""

# =========================
# DB CONNECTION
# =========================
db_connection = mysql.connector.connect(
    host="68.178.155.255",
    user="Anika12",
    password="Anika12",
    database="categories_lead"
)
cursor = db_connection.cursor(dictionary=True)

# =========================
# MAIN LOGIC: CALL SENTIMENT
# =========================
cursor.execute("SELECT DISTINCT lead_id FROM calls WHERE lead_id IS NOT NULL")
lead_ids = cursor.fetchall()

for row in lead_ids:
    lead_id = row['lead_id']

    # Fetch latest call per lead by combining call_date + call_time
    cursor.execute("""
        SELECT call_result, description
        FROM calls
        WHERE lead_id = %s
        ORDER BY STR_TO_DATE(CONCAT(call_date, ' ', call_time), '%%Y-%%m-%%d %%h:%%i %%p') DESC
        LIMIT 1
    """, (lead_id,))
    latest_call = cursor.fetchone()

    if latest_call:
        prompt = build_negative_sentiment_flag_prompt_for_calls(latest_call)
        output = run_model(prompt)

        print(f"☎️ Lead {lead_id}")
        print(f"📞 call_result: {latest_call['call_result']}")
        print(f"📝 description: {latest_call['description']}")
        print(f"🧠 Model Output: {output}")

        # Update calls_category (1 for True, 0 for False)
        cursor.execute("""
            UPDATE lead_engagement_summary
            SET calls_category = %s
            WHERE lead_id = %s
        """, (1 if output.lower() == "true" else 0, lead_id))

        print(f"✅ Updated calls_category for lead {lead_id} → {output}")
    else:
        print(f"⚠️ No call found for lead {lead_id}")

# =========================
# FINALIZE
# =========================
db_connection.commit()
cursor.close()
db_connection.close()
print("\n✅ All call-based sentiment updates complete in lead_engagement_summary.")


In [ ]:
# meetings
from datetime import datetime
import mysql.connector
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# =========================
# MODEL SETUP
# =========================
model_name = "google/flan-t5-base"  # or flan-t5-xl if you prefer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

def run_model(prompt):
    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=10)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        return result if result else "False"
    except Exception as e:
        print(f"⚠️ Model error: {e}")
        return "False"

# =========================
# PROMPT FUNCTION
# =========================
def build_negative_sentiment_flag_prompt_for_meetings(latest_meeting_description):
    return f"""
You are a CRM assistant.

Your task is to **analyze this meeting description** and check if it shows **any sign of negative sentiment**, based on these 3 types:

🔹 Complaint – about service, delays, lack of follow-up, or poor experience  
🔹 Frustration – irritated tone, repeated follow-ups, impatience  
🔹 Negative Product/Service Feedback – bugs, integration issues, not working, too expensive

✅ If **any one** of the above is present in the meeting description, return: **True**  
❌ If none are found, return: **False**

---

Examples:

1.  
📝 description: "Client expressed frustration that no one joined the demo as planned."  
→ Complaint + Frustration  
Return: True

2.  
📝 description: "Lead mentioned the CRM setup has several bugs and delays."  
→ Negative Product Feedback  
Return: True

3.  
📝 description: "Demo went well, no concerns raised."  
→ No negative signal  
Return: False

4.  
📝 description: "Client was impatient and said the onboarding is taking too long."  
→ Complaint + Frustration  
Return: True

---

Now analyze the meeting description below and return only `True` or `False`:

📝 description: {latest_meeting_description}

Answer:
"""

# =========================
# DB CONNECTION
# =========================
db_connection = mysql.connector.connect(
    host="68.178.155.255",
    user="Anika12",
    password="Anika12",
    database="categories_lead"
)
cursor = db_connection.cursor(dictionary=True)

# =========================
# MAIN LOGIC: MEETING SENTIMENT
# =========================
cursor.execute("SELECT DISTINCT related_to as lead_id FROM meetings WHERE related_to IS NOT NULL")
lead_ids = cursor.fetchall()

for row in lead_ids:
    lead_id = row['lead_id']

    # Fetch latest meeting based on meeting_datetime
    cursor.execute("""
        SELECT description
        FROM meetings
        WHERE related_to = %s
        ORDER BY STR_TO_DATE(meeting_datetime, '%%Y-%%m-%%dT%%H:%%i') DESC
        LIMIT 1
    """, (lead_id,))
    latest_meeting = cursor.fetchone()

    if latest_meeting and latest_meeting['description']:
        prompt = build_negative_sentiment_flag_prompt_for_meetings(latest_meeting['description'])
        output = run_model(prompt)

        print(f"📅 Lead {lead_id}")
        print(f"📝 description: {latest_meeting['description']}")
        print(f"🧠 Model Output: {output}")

        # Update meetings_category in lead_engagement_summary
        cursor.execute("""
            UPDATE lead_engagement_summary
            SET meetings_category = %s
            WHERE lead_id = %s
        """, (1 if output.lower() == "true" else 0, lead_id))

        print(f"✅ Updated meetings_category for lead {lead_id} → {output}")
    else:
        print(f"⚠️ No valid meeting found for lead {lead_id}")

# =========================
# FINALIZE
# =========================
db_connection.commit()
cursor.close()
db_connection.close()
print("\n✅ All meeting-based sentiment updates complete in lead_engagement_summary.")


In [ ]:
#inactivity_days, emails_catgeory,calls_catgeory,meetings_category 

from datetime import datetime
import mysql.connector
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# =========================
# MODEL SETUP
# =========================
model_name = "google/flan-t5-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

def run_model(prompt):
    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=10)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        return result if result else "False"
    except Exception as e:
        print(f"⚠️ Model error: {e}")
        return "False"

# =========================
# PROMPT FUNCTIONS
# =========================
def build_email_prompt(convo):
    return f"""
You are a CRM assistant.

Your task is to **analyze this lead conversation** and check if it shows **any sign of negative sentiment**, based on these 3 types:

🔹 Complaint – about service, delays, lack of follow-up, or experience  
🔹 Frustration – irritated tone, repeated follow-ups, impatience  
🔹 Negative Product/Service Feedback – bugs, crashes, not working, too expensive

✅ If **any one** of the above is present in the conversation, return: **True**  
❌ If none are found, return: **False**

---
{convo}
Answer:
"""

def build_call_prompt(call):
    return f"""
You are a CRM assistant.

Your task is to **analyze this latest call summary** and check if it shows **any sign of negative sentiment**, based on these 3 types:

🔹 Complaint – about service, delays, lack of follow-up, or poor experience  
🔹 Frustration – irritated tone, repeated follow-ups, impatience  
🔹 Negative Product/Service Feedback – bugs, integration issues, not working, too expensive

✅ If **any one** of the above is present in the call summary, return: **True**  
❌ If none are found, return: **False**

---
📞 call_result: {call.get('call_result', 'N/A')}  
📝 description: {call.get('description', 'N/A')}
Answer:
"""

def build_meeting_prompt(description):
    return f"""
You are a CRM assistant.

Your task is to **analyze this meeting description** and check if it shows **any sign of negative sentiment**, based on these 3 types:

🔹 Complaint – about service, delays, lack of follow-up, or poor experience  
🔹 Frustration – irritated tone, repeated follow-ups, impatience  
🔹 Negative Product/Service Feedback – bugs, integration issues, not working, too expensive

✅ If **any one** of the above is present in the meeting description, return: **True**  
❌ If none are found, return: **False**

---
📝 description: {description}
Answer:
"""

# =========================
# DB CONNECTION
# =========================
db = mysql.connector.connect(
    host="68.178.155.255",
    user="Anika12",
    password="Anika12",
    database="categories_lead"
)
cursor = db.cursor(dictionary=True)

# =========================
# GET DISTINCT LEADS
# =========================
cursor.execute("""
    SELECT DISTINCT lead_id FROM (
        SELECT lead_id FROM emails
        UNION
        SELECT lead_id FROM calls
        UNION
        SELECT related_to AS lead_id FROM meetings
    ) AS all_leads
""")
leads = cursor.fetchall()

# =========================
# PROCESS EACH LEAD
# =========================
for lead in leads:
    lead_id = lead['lead_id']
    print(f"\n🔄 Processing lead: {lead_id}")

    # -------------------------
    # EMAILS: sentiment + date
    # -------------------------
    cursor.execute("""
        SELECT direction, body, timestamp
        FROM emails
        WHERE lead_id = %s
        ORDER BY timestamp DESC
        LIMIT 5
    """, (lead_id,))
    emails = list(reversed(cursor.fetchall()))
    email_neg_count = 0
    latest_email_time = None
    i = 0
    while i < len(emails) - 1:
        sent = emails[i]
        received = emails[i + 1]
        if sent['direction'] == 'sent' and received['direction'] == 'received':
            convo = f"sent: {sent['body'].strip()}\nreceived: {received['body'].strip()}"
            prompt = build_email_prompt(convo)
            output = run_model(prompt)
            if output.lower() == 'true':
                email_neg_count += 1
            i += 2
        else:
            i += 1
    if emails:
        latest_email_time = max(e['timestamp'] for e in emails if e['timestamp'])

    # -------------------------
    # CALLS: sentiment + date
    # -------------------------
    cursor.execute("""
        SELECT call_result, description, call_date
        FROM calls
        WHERE lead_id = %s
        ORDER BY STR_TO_DATE(call_date, '%%Y-%%m-%%d') DESC
        LIMIT 1
    """, (lead_id,))
    call = cursor.fetchone()
    calls_category = 0
    latest_call_time = None
    if call:
        prompt = build_call_prompt(call)
        output = run_model(prompt)
        if output.lower() == "true":
            calls_category = 1
        if call['call_date']:
            latest_call_time = datetime.strptime(call['call_date'], "%Y-%m-%d")

    # -------------------------
    # MEETINGS: sentiment + date
    # -------------------------
    cursor.execute("""
        SELECT description, meeting_datetime
        FROM meetings
        WHERE related_to = %s
        ORDER BY STR_TO_DATE(meeting_datetime, '%%Y-%%m-%%dT%%H:%%i') DESC
        LIMIT 1
    """, (lead_id,))
    meeting = cursor.fetchone()
    meetings_category = 0
    latest_meeting_time = None
    if meeting and meeting['description']:
        prompt = build_meeting_prompt(meeting['description'])
        output = run_model(prompt)
        if output.lower() == "true":
            meetings_category = 1
        if meeting['meeting_datetime']:
            latest_meeting_time = datetime.strptime(meeting['meeting_datetime'], "%Y-%m-%dT%H:%M")

    # -------------------------
    # CALCULATE INACTIVITY DAYS
    # -------------------------
    latest_dates = [d for d in [latest_email_time, latest_call_time, latest_meeting_time] if d]
    inactivity_days = (datetime.now() - max(latest_dates)).days if latest_dates else None

    # -------------------------
    # UPDATE SUMMARY TABLE
    # -------------------------
    cursor.execute("""
        UPDATE lead_engagement_summary
        SET inactivity_days = %s,
            emails_category = %s,
            calls_category = %s,
            meetings_category = %s
        WHERE lead_id = %s
    """, (
        inactivity_days,
        email_neg_count,
        calls_category,
        meetings_category,
        lead_id
    ))

    print(f"📊 Updated ➤ inactivity_days: {inactivity_days}, emails: {email_neg_count}, calls: {calls_category}, meetings: {meetings_category}")

# =========================
# COMMIT & CLOSE
# =========================
db.commit()
cursor.close()
db.close()
print("\n✅ All updates complete.")


In [ ]:
#total interactions week wise
import mysql.connector
from datetime import datetime
import json

# ---------------------------
# Database Connection
# ---------------------------
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="categories_lead"
    )

# ---------------------------
# Utility: Week Binning
# ---------------------------
def bin_weekly(entries, start_date):
    weekly_counts = {}
    for time in entries:
        delta = (time - start_date).days
        week_num = max((delta // 7) + 1, 1)
        key = f"week {week_num}"
        weekly_counts[key] = weekly_counts.get(key, 0) + 1
    return weekly_counts

def fill_missing_weeks(*dicts):
    max_week = 0
    for d in dicts:
        if d:
            max_week = max(max_week, max(int(k.split()[1]) for k in d))
    filled = {}
    for i in range(1, max_week + 1):
        filled[f"week {i}"] = 0
    for d in dicts:
        for k, v in d.items():
            filled[k] += v
    return dict(sorted(filled.items(), key=lambda x: int(x[0].split()[1])))

# ---------------------------
# Main Logic
# ---------------------------
def calculate_total_interactions():
    db = connect_db()
    cursor = db.cursor(dictionary=True)

    # Fetch all leads and their created_at
    cursor.execute("SELECT lead_id, created_at FROM leads")
    leads = cursor.fetchall()

    for lead in leads:
        lead_id = lead['lead_id']
        created_at = lead['created_at']
        if isinstance(created_at, str):
            created_at = datetime.strptime(created_at, "%Y-%m-%d %H:%M:%S")

        # Get email timestamps
        cursor.execute("""
            SELECT timestamp FROM emails
            WHERE lead_id = %s AND timestamp IS NOT NULL
        """, (lead_id,))
        email_entries = [
            datetime.strptime(row['timestamp'], "%Y-%m-%d %H:%M:%S")
            for row in cursor.fetchall()
        ]
        email_weeks = bin_weekly(email_entries, created_at)

        # Get call timestamps
        cursor.execute("""
            SELECT created_at FROM calls
            WHERE lead_id = %s AND created_at IS NOT NULL
        """, (lead_id,))
        call_entries = [row['created_at'] for row in cursor.fetchall()]
        call_weeks = bin_weekly(call_entries, created_at)

        # Get meeting timestamps
        cursor.execute("""
            SELECT created_at FROM meetings
            WHERE related_to = %s AND created_at IS NOT NULL
        """, (lead_id,))
        meeting_entries = [row['created_at'] for row in cursor.fetchall()]
        meeting_weeks = bin_weekly(meeting_entries, created_at)

        # Combine all into total_interactions
        total_weeks = fill_missing_weeks(email_weeks, call_weeks, meeting_weeks)
        total_json = json.dumps(total_weeks)

        # Update lead_engagement_summary
        cursor.execute("""
            UPDATE lead_engagement_summary
            SET total_interactions = %s
            WHERE lead_id = %s
        """, (total_json, lead_id))

        print(f"✅ Lead {lead_id} → {total_json}")

    db.commit()
    db.close()

# ---------------------------
# Run it
# ---------------------------
calculate_total_interactions()


In [ ]:
#logic to predict churn or not
import json
import mysql.connector

def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="categories_lead"
    )

def fetch_all_leads():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        cursor.execute("""
            SELECT lead_id, inactivity_days, negative_sentiment_count, total_interactions
            FROM lead_engagement_summary
        """)
        results = cursor.fetchall()
        conn.close()
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return []

def update_churn_label(lead_id, churn_score, churn_label):
    try:
        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE lead_engagement_summary
            SET churn_score = %s,
                churn_label = %s
            WHERE lead_id = %s
        """, (churn_score, churn_label, lead_id))
        conn.commit()
        conn.close()
    except mysql.connector.Error as err:
        print(f"Error updating lead {lead_id}: {err}")

def calculate_churn_score(lead):
    score = 0

    # 1. Negative Sentiment Count (Max 30)
    nsc = lead.get('negative_sentiment_count') or 0
    if nsc == 1:
        score += 10
    elif nsc == 2:
        score += 20
    elif nsc >= 3:
        score += 30

    # 2. Inactivity Days (Max 35)
    try:
        inactivity = int(lead.get('inactivity_days') or 0)
    except:
        inactivity = 0

    if inactivity >= 30:
        score += 35
    else:
        score += (inactivity / 30) * 35

    # 3. Total Interactions Week 4+ (Max 35)
    total_json = lead.get('total_interactions') or '{}'
    bad_weight = 0
    total_weight = 0
    try:
        interaction_dict = json.loads(total_json)
        for week_key, count in interaction_dict.items():
            if week_key.startswith("week"):
                week_num = int(week_key.split()[1])
                if week_num >= 4:
                    weight = week_num - 3
                    total_weight += weight
                    if count <= 2:
                        bad_weight += weight
    except Exception as e:
        print(f"Error decoding total_interactions for lead {lead['lead_id']}: {e}")

    if total_weight > 0:
        score += (bad_weight / total_weight) * 35

    # Cap score
    score = round(min(score, 100), 2)

    # Label
    if score > 70:
        label = "High Churn Risk"
    elif score >= 40:
        label = "Medium Churn Risk"
    else:
        label = "Low Churn Risk"

    return score, label

# 🚀 Execute for all leads
leads = fetch_all_leads()

for lead in leads:
    score, label = calculate_churn_score(lead)
    update_churn_label(lead['lead_id'], score, label)
    print(f"Lead {lead['lead_id']} → Score: {score}, Label: {label}")
